본 파일은 알바몬 페이지를 크롤링 하기 위함임..
https://www.albamon.com/list/gi/mon_area_list.asp?page=2&ps=20&ob=6&lvtype=1&rArea=,B210&rWDate=1&Empmnt_Type=

이게 링크이며 이를 토대로 크롤링 진행!

In [3]:

from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
import re
import requests

session = requests.Session()

title = [] #알바title = [] #알바 제목

result = pd.DataFrame(columns=['지역', '근무회사', '근무시간','급여','올린시간','알바설명'])  # 결과저장용 DataFrame
new_df = pd.DataFrame(columns=['지역', '근무회사', '근무시간','급여','올린시간','알바설명'])  # append용 DataFrame


pagenum = 1 #첫 페이지
while(len(result) <100):  #n개 페이지에 대하여 크롤링
    print(pagenum)
    
    # 리스트 초기화
    area_list = []
    cName_list = []
    cTit_list = []
    pay_list = []
    time_list = []
    recently_list = []
    url = 'https://www.albamon.com/list/gi/mon_area_list.asp?page=argu&ps=20&ob=6&lvtype=1&rArea=,B210&rWDate=1&Empmnt_Type=' #이건 알바천국
    url = url.replace('argu', str(pagenum))  # 페이지 1에대한 경우 이후 이는 변수로 처리할 것.

    pagenum = pagenum + 1 #다음페이지 준비
    page = session.get(url)
    c = page.content
    soup = BeautifulSoup(c, 'html.parser')
    title_ = soup.find_all('td')
    ##form > div.listType-normal > div.gListWrap > table > tbody
    conte = soup.select_one('#form > div.listType-normal > div.gListWrap > table > tbody')
    
    #지역관련 크롤링
    area = conte.select('td.area')
    for ww in area:
        area_list.append(ww.get_text().strip().replace('스크랩\n',"")) #지역리스트 데이터 추가함
        #print(ww.get_text().strip().replace('스크랩\n',""))
        
    #==================================================================

    #알바 제목
    cName = conte.select('td.subject > div.subWrap > p.cName')
    for ww in cName:
        cName_list.append(ww.get_text().replace('\n',''))
        #print(ww.get_text())

    #==================================================================
    #내용

    cTit = conte.select('td.subject > div.subWrap > p.cTit')
    for ww in cTit:
        cTit_list.append(ww.get_text())
        #print(ww.get_text())
 


    #===================================================
    #급여
    pay = conte.select('td.pay')
    #지급방식은 돈의 범위로 한다.
    #시급 일급 월급으로 
    for ww in pay:
        pay_list.append(ww.get_text().strip())
        #print(ww.get_text().strip())


    #==================================================
    #근무시간
    time = conte.select('td:nth-child(4)')
    for ww in time:
        time_list.append(ww.get_text().strip())
        #print(ww.get_text().strip())


    #================================================
    #올린시간

    #태그의 변화 날짜는 못받아오더라
    recently_ = conte.select('td.recently')
    print(recently_)
    for ws in recently_:
        recently_list.append(ws.get_text())

    print(recently_list)

    
    new_df['지역'] = area_list
    new_df['근무회사'] = cName_list
    new_df['근무시간'] = time_list
    new_df['급여'] = pay_list
    print(url)
    new_df['올린시간'] = recently_list
    new_df['알바설명'] = cTit_list    
    #print(new_df)


    result = pd.concat([result,new_df])
    result = result.drop_duplicates(['근무회사'], keep='first')  # 중복글에 대한 처리
    
    print(result)
    
    result['순서'] = range(len(result))
    result = result.set_index('순서')
    print(len(result))
    
    #여기서 급여 190만원 이상되는 알바는 컷 해야함.
    #먼저 급여에서 원 문자와 ,문자를 제거하고 순수하게 정수형으로 제어한다.
    #그러려먼 먼저 result 데이터 프레임 상에서 형을 바꿔줘야 하는데. 이거 문제가 많네
result.to_csv('albamon.csv', encoding='CP949', index=False)


print(result)


1
[<td class="recently"><em>20분전</em></td>, <td class="recently"><em>22분전</em></td>, <td class="recently"><em>26분전</em></td>, <td class="recently"><em>33분전</em></td>, <td class="recently"><em>33분전</em></td>, <td class="recently"><em>36분전</em></td>, <td class="recently"><em>38분전</em></td>, <td class="recently"><em>50분전</em></td>, <td class="recently"><em>50분전</em></td>, <td class="recently"><em>51분전</em></td>, <td class="recently"><em>1시간전</em></td>, <td class="recently"><em>1시간전</em></td>, <td class="recently"><em>1시간전</em></td>, <td class="recently"><em>1시간전</em></td>, <td class="recently"><em>1시간전</em></td>, <td class="recently"><em>1시간전</em></td>, <td class="recently"><em>1시간전</em></td>, <td class="recently"><em>1시간전</em></td>, <td class="recently"><em>1시간전</em></td>, <td class="recently"><em>1시간전</em></td>]
['20분전', '22분전', '26분전', '33분전', '33분전', '36분전', '38분전', '50분전', '50분전', '51분전', '1시간전', '1시간전', '1시간전', '1시간전', '1시간전', '1시간전', '1시간전', '1시간전', '1시간전', '1시간전']
https://www.alba

[<td class="recently"><em>2시간전</em></td>, <td class="recently"><em>2시간전</em></td>, <td class="recently"><em>2시간전</em></td>, <td class="recently"><em>2시간전</em></td>, <td class="recently"><em>2시간전</em></td>, <td class="recently"><em>2시간전</em></td>, <td class="recently"><em>2시간전</em></td>, <td class="recently"><em>2시간전</em></td>, <td class="recently"><em>2시간전</em></td>, <td class="recently"><em>2시간전</em></td>, <td class="recently"><em>3시간전</em></td>, <td class="recently"><em>4시간전</em></td>, <td class="recently"><em>16시간전</em></td>, <td class="recently"><em>17시간전</em></td>, <td class="recently"><em>17시간전</em></td>, <td class="recently"><em>17시간전</em></td>, <td class="recently"><em>17시간전</em></td>, <td class="recently"><em>17시간전</em></td>, <td class="recently"><em>17시간전</em></td>, <td class="recently"><em>17시간전</em></td>]
['2시간전', '2시간전', '2시간전', '2시간전', '2시간전', '2시간전', '2시간전', '2시간전', '2시간전', '2시간전', '3시간전', '4시간전', '16시간전', '17시간전', '17시간전', '17시간전', '17시간전', '17시간전', '17시간전', '17시간전']
ht

[<td class="recently">06/25</td>, <td class="recently">06/25</td>, <td class="recently">06/24</td>, <td class="recently">06/24</td>, <td class="recently">06/24</td>, <td class="recently">06/24</td>, <td class="recently">06/23</td>, <td class="recently">06/22</td>, <td class="recently">06/22</td>, <td class="recently">06/21</td>, <td class="recently"><em>45분전</em></td>, <td class="recently"><em>2시간전</em></td>, <td class="recently"><em>2시간전</em></td>, <td class="recently"><em>3시간전</em></td>, <td class="recently"><em>4시간전</em></td>, <td class="recently"><em>4시간전</em></td>, <td class="recently"><em>5시간전</em></td>, <td class="recently"><em>5시간전</em></td>, <td class="recently"><em>6시간전</em></td>, <td class="recently"><em>6시간전</em></td>]
['06/25', '06/25', '06/24', '06/24', '06/24', '06/24', '06/23', '06/22', '06/22', '06/21', '45분전', '2시간전', '2시간전', '3시간전', '4시간전', '4시간전', '5시간전', '5시간전', '6시간전', '6시간전']
https://www.albamon.com/list/gi/mon_area_list.asp?page=5&ps=20&ob=6&lvtype=1&rArea=,B210

In [11]:
result["급여"] = result["급여"].str.replace('원','')
result["급여"] = result["급여"].str.replace(',','')

In [16]:
result = result.astype({"급여":int})

In [17]:
if(result['급여'] < 40000 ): result['급여방식'] = '시급' 

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().